In [1]:
import os
import pandas as pd
os.getcwd()

'C:\\Users\\regis\\OneDrive\\Dokumente\\GitHub\\wake_effect\\delta_wind_velocity'

In [2]:
wind_dir = pd.read_csv('C:/Users/regis/OneDrive/Dokumente/GitHub/wake_effect/data/Wind_direction.csv')
wind_v = pd.read_csv('C:\\Users\\regis\\OneDrive\\Dokumente\\GitHub\\wake_effect\\data\\Windfarm.csv')

In [3]:
wind_dir['60.0'].isna().sum()

15084

In [4]:
df = wind_dir.merge(wind_v, left_on='Unnamed: 0', right_on='Unnamed: 0')

In [5]:
df

,Unnamed: 0,10.0,40.0,60.0,80.0,100.0,WTG1,WTG2,WTG3,WTG4,...,WTG23,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32
0,2013-08-01 00:00:00,NaN,NaN,129.7,110.8,113.0,11.3,12.4,10.4,9.8,...,13.4,12.5,12.2,12.6,12.5,12.8,13.8,12.9,12.4,8.9
1,2013-08-01 00:10:00,NaN,NaN,111.9,113.0,115.5,11.6,11.9,10.4,10.7,...,12.7,13.0,12.5,14.4,12.4,12.8,13.4,13.1,13.2,9.8
2,2013-08-01 00:20:00,NaN,NaN,116.3,110.2,112.2,11.8,12.5,10.8,10.6,...,12.2,12.1,11.7,12.2,12.0,12.2,13.2,12.4,12.3,8.7
3,2013-08-01 00:30:00,NaN,NaN,115.2,112.6,115.9,11.7,11.6,10.6,10.4,...,12.8,13.2,10.9,12.4,12.3,12.3,12.4,11.8,10.5,8.6
4,2013-08-01 00:40:00,NaN,NaN,118.4,116.6,118.8,10.5,11.0,8.6,8.8,...,12.6,12.7,10.9,11.9,12.0,12.5,10.0,12.0,11.5,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,2014-07-31 23:10:00,NaN,NaN,NaN,99.8,101.8,8.5,8.9,7.5,7.4,...,9.3,9.6,8.9,9.9,9.6,9.7,9.3,9.4,9.2,5.1
52556,2014-07-31 23:20:00,NaN,NaN,NaN,103.0,106.0,8.3,9.3,7.6,7.3,...,9.2,9.6,9.5,9.5,9.1,9.4,9.3,9.9,9.6,5.1
52557,2014-07-31 23:30:00,NaN,NaN,NaN,104.5,106.9,9.5,9.4,8.2,7.7,...,9.6,10.1,9.5,10.1,9.7,9.9,9.7,10.0,9.8,5.4
52558,2014-07-31 23:40:00,NaN,NaN,NaN,106.4,108.5,9.5,9.8,8.3,7.6,...,10.0,9.9,9.7,10.0,10.0,10.2,9.8,10.2,9.9,5.3


In [14]:
df['60.0'] = df['60.0'].round(decimals=0)
df['80.0'] = df['80.0'].round(decimals=0)
df['100.0'] = df['100.0'].round(decimals=0)
df = df.drop(columns = ['10.0'])
df = df.drop(columns = ['40.0'])

KeyError: "['10.0'] not found in axis"

In [27]:
df1 = df.groupby(['100.0']).mean()

In [28]:
df11 = df1['WTG3']

In [29]:
//// test

SyntaxError: invalid syntax (<ipython-input-29-f5f40dfd27e9>, line 1)

In [52]:
df2 = df.groupby(['80.0']).mean()

In [53]:
df2

,60.0,100.0,WTG1,WTG2,WTG3,WTG4,WTG5,WTG6,WTG7,WTG8,...,WTG23,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32
80.0,,,,,,,,,,,,,,,,,,,,,
0.0,9.00,5.000000,3.150000,3.100000,3.050000,2.550000,2.900000,2.500000,3.25,2.30,...,3.700000,2.950000,3.850000,2.950000,3.85,3.150000,3.750000,4.250000,4.05,3.750000
1.0,19.00,22.000000,3.200000,3.200000,2.900000,2.800000,2.900000,2.500000,3.10,2.70,...,1.700000,2.000000,2.000000,2.200000,2.10,2.000000,2.200000,2.500000,2.70,2.000000
2.0,353.00,7.000000,5.100000,5.000000,5.400000,4.900000,5.200000,4.900000,5.30,4.50,...,5.000000,4.400000,5.100000,4.000000,5.10,4.000000,5.100000,5.600000,5.30,5.000000
3.0,4.00,18.500000,2.700000,2.550000,2.700000,2.550000,2.600000,2.500000,2.80,2.50,...,1.600000,1.650000,1.450000,1.750000,2.05,1.550000,1.250000,1.000000,1.40,1.300000
4.0,NaN,16.000000,2.700000,2.600000,3.000000,3.300000,3.100000,2.900000,2.90,2.70,...,1.900000,2.200000,2.300000,2.400000,1.70,1.700000,1.700000,2.000000,2.20,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356.0,350.00,13.000000,1.750000,1.750000,1.500000,1.550000,1.650000,1.300000,1.65,1.25,...,2.300000,2.050000,2.700000,1.700000,2.45,1.750000,2.200000,2.400000,2.50,2.500000
357.0,177.75,97.750000,4.250000,4.275000,4.575000,4.225000,4.250000,3.825000,4.25,3.90,...,4.075000,3.475000,4.100000,3.175000,4.15,3.050000,3.875000,4.500000,4.35,4.150000
358.0,7.00,353.000000,1.200000,1.400000,1.200000,1.100000,1.900000,1.200000,0.90,1.10,...,1.000000,1.200000,0.800000,1.300000,1.20,0.900000,1.200000,1.200000,1.60,0.900000


In [31]:
df3 = df1['WTG3'] - df2['WTG3']


In [54]:
df.sum('80.0')

ValueError: No axis named 80.0 for object type <class 'pandas.core.frame.DataFrame'>

In [32]:
for i in df3:
    print(i)

nan
2.0749999999999997
nan
-0.49999999999999956
2.9000000000000004
nan
nan
3.000000000000001
nan
-2.5666666666666673
nan
-2.1333333333333333
3.4750000000000005
1.2500000000000009
-0.326666666666668
-1.2750000000000004
0.04285714285714359
-0.3285714285714274
1.15
-1.2333333333333334
1.212499999999999
0.05714285714285783
-0.47499999999999964
-0.24666666666666703
-0.44571428571428573
-0.1437500000000016
-1.4583333333333321
0.40756302521008436
-0.44545454545454444
-0.32414860681114543
-0.5216666666666665
-0.8869281045751629
0.18644501278772196
0.3599537037037024
0.38395802098950416
-0.9779354838709677
-0.6121693121693133
-0.6542597187758483
0.166187739463604
-0.06971514242878563
-0.19142857142857306
-0.23831199620673438
0.07511784511784647
0.011538461538464162
-0.09711607786589926
-0.21831306990881494
-0.16420512820512467
0.04096454441479391
0.41703762586115545
-0.4237704918032792
-0.42577824049973767
0.09763087843833596
-0.2924528301886786
-0.07525900025900256
-0.08871090770404333
-0.0615

In [45]:
x['id_sum'] = df.groupby('id')['value'].transform('sum')

TypeError: 'method' object is not subscriptable

In [ ]:
x